In [1]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [2]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'],})
        nochild.update({key : filnary})

In [3]:
nary['GO:0000015']

{'id': 'GO:0000015',
 'isObsolete': False,
 'name': 'phosphopyruvate hydratase complex',
 'definition': {'text': 'A multimeric enzyme complex, usually a dimer or an octamer, that catalyzes the conversion of 2-phospho-D-glycerate to phosphoenolpyruvate and water.',
  'xrefs': [{'dbCode': 'ISBN', 'dbId': '0198506732'}]},
 'synonyms': [{'name': 'enolase complex', 'type': 'exact'}],
 'aspect': 'cellular_component',
 'usage': 'Unrestricted'}

In [3]:
#get parents

for key, value in kilnary.items():
    for item in value['children']:
        if nary[item['id']].get('parent') == None:
            nary[item['id']]['parent'] = []
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
        else:
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
            
for key, value in nary.items():
    if value.get('parent') == None:
        value['parent'] = []

In [4]:
#empty children class for nochilds

for key, value in nary.items():
    if value.get('children') == None:
        value['children'] = []

In [5]:
for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                print(item)

{'id': 'GO:0016047', 'relation': 'is_a'}
{'id': 'GO:0009603', 'relation': 'is_a'}


In [6]:
#get rid of the two motherfuckers

for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                del(v['parent'][v['parent'].index(item)])
                del(v['children'][v['children'].index(item)])

In [7]:
#remove anything that is not a biological_process or molecular_function

c = 0
keys = []
for key, value in nary.items():
    if value['aspect'] != 'biological_process' and value['aspect'] != 'molecular_function':
        keys.append(key)
        c+=1
print(c)
for i in keys:
    del nary[i]

4367


In [8]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [9]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto
    class biological_process_main(Thing):
        namespace = onto
    class molecular_function_main(Thing):
        namespace = onto
    class drug(Thing):
        namespace = onto

In [12]:
def recursive_classmaker_down(term):
    for item in nary[term]['children']:
        if item['relation'] == 'is_a':
            with onto:
                nc = types.new_class(('G' + item['id'][3:]), (getattr(onto, ('G' +term[3:])),))
            recursive_classmaker_down(item['id'])
    return 0

In [13]:
#Using classmaker to make FTC classes

for key, value in nary.items():
    if value['parent'] == [] and value['aspect'] == 'biological_process':
        with onto:
            nc = types.new_class(('G' + key[3:]), (biological_process_main,))
        for item in value['children']:
            if item['relation'] == 'is_a':
                with onto:
                    nc = types.new_class(('G' + item['id'][3:]), (getattr(onto, ('G' + key[3:])),))
                recursive_classmaker_down(item['id'])
    if value['parent'] == [] and value['aspect'] == 'molecular_function':
        with onto:
            nc = types.new_class(('G' + key[3:]), (molecular_function_main,))
        for item in value['children']:
            if item['relation'] == 'is_a':                
                with onto:
                    nc = types.new_class(('G' + item['id'][3:]), (getattr(onto, ('G' + key[3:])),))
                recursive_classmaker_down(item['id'])
        


In [21]:
#list of classes created

GO_classes_created = [str(x)[5:] for x in list(onto.classes())]
len(GO_classes_created)

42987

In [20]:
for x in GO_classes_created:
    if x[2] != ':':
        print(x)

therapeutic_agent
biological_process_main
molecular_function_main
drug
GGO:0003674
GGO:0045182
G0090079
G0003674
G0045182
G0000900
G0008494
G0008135
G0003743
G0008079
G0003747
G0016150
G0016149
G0003746
G0004694
G0030371
G0000901
G0045183
G0140110
G0001072
G0003711
G0003700
G0001217
G0001227
G0001216
G0001228
G0098531
G0004879
G0000981
G0034246
G0043856
G0001073
G0140223
G0001181
G0000995
G0016251
G0001070
G0001134
G0001186
G0001004
G0001135
G0016987
G0003712
G0003713
G0030374
G0030375
G0003714
G0016989
G0140313
G0140315
G0140319
G0140320
G0140314
G0140311
G0140104
G0000036
G0036370
G1904091
G0016530
G0034986
G0016531
G0016532
G0032977
G0097163
G0140132
G0044183
G0098772
G0030234
G0043763
G0030337
G0019207
G0019210
G0004860
G0030292
G0030294
G0030293
G0008427
G0030291
G0008426
G0004862
G0004861
G0010307
G0019887
G0016538
G0061575
G0042556
G0042557
G0008607
G0008603
G0019914
G0030295
G0030296
G0030298
G0030297
G0019912
G0043539
G0010858
G0019209
G0098744
G0035014
G0046935
G0010851
G0010

G0050599
G0102717
G0018585
G0008388
G0034925
G0061897
G0016716
G0018686
G0004503
G0030338
G0080106
G0016712
G0047646
G0070330
G0140399
G0036383
G0102977
G0008726
G0062047
G0052881
G0034938
G0080102
G0102125
G0046996
G0050046
G0047056
G0102963
G0047052
G0047053
G0047054
G0047055
G0102632
G0018596
G0008399
G0008681
G0050602
G0004508
G0052637
G0018591
G0008390
G0018679
G0050016
G0050207
G0044876
G0018588
G0050012
G0001567
G0019135
G0008387
G0034903
G0043823
G0008401
G0062182
G0061898
G0018689
G0080107
G0016717
G0102988
G0045300
G0102654
G0102656
G0102839
G0102835
G0102856
G0102837
G0102858
G0102852
G0102854
G0102875
G0102873
G0102850
G0102871
G0102431
G0102993
G0102849
G0102845
G0000248
G0102847
G0102841
G0102866
G0102864
G0102843
G0102862
G0102860
G0102985
G0102987
G0102655
G0102676
G0102657
G0102838
G0102834
G0102857
G0102836
G0102859
G0102853
G0102855
G0102874
G0102872
G0102851
G0102430
G0102846
G0102848
G0000249
G0102865
G0102842
G0102863
G0102844
G0016215
G0016213
G0004768
G0032896
G

G0047718
G0103081
G0047722
G0102722
G0047036
G0033707
G0103066
G0050085
G0008912
G0052660
G0052661
G0016617
G0052747
G0099618
G0102176
G0030267
G0089714
G0047941
G0047003
G0018453
G0004455
G0050090
G0047049
G0047964
G0050574
G0050009
G0019151
G0050256
G0047838
G0047041
G0050037
G0033845
G0048258
G0000253
G0047109
G0047903
G0102187
G0004745
G0102394
G0047502
G0050388
G0033711
G0009026
G0047952
G0004367
G0042356
G0050123
G0047014
G0047037
G0018464
G0033708
G0009040
G0103067
G0050086
G0051903
G0004420
G0008743
G0050229
G0033833
G0050221
G0008679
G0102198
G0004757
G0047880
G0030266
G0047559
G0032442
G0103093
G0102538
G0102175
G0047027
G0045551
G0103030
G0047004
G0018452
G0102069
G0050573
G0047837
G0050038
G0102566
G0047890
G0047501
G0102182
G0047038
G0004448
G0004449
G0004450
G0102976
G0103087
G0102539
G0047015
G0018463
G0047743
G0047995
G0033705
G0102641
G0047030
G0000140
G0050606
G0050268
G0003938
G0004090
G0102072
G0050491
G0102340
G0047550
G0103090
G0008678
G0047512
G0050109
G0036132
G

G0050106
G0033889
G0033888
G0033887
G0033886
G0047825
G0003943
G0034889
G0045330
G0052689
G0080031
G0047845
G0046555
G0047709
G0050528
G0004806
G0004465
G0035460
G0047593
G0018731
G0033879
G0030600
G0047570
G0004453
G0102257
G0102567
G0047739
G0047856
G0052739
G0047610
G0047374
G0047378
G0102355
G0050389
G0047989
G0080032
G0050185
G0047746
G0050525
G0050318
G0050529
G0050398
G0018734
G0050181
G0052793
G0052797
G0004104
G0003990
G0050253
G1990699
G0047375
G0047908
G0004622
G0102543
G0047583
G0047745
G0034338
G0008126
G0046575
G0001681
G0004341
G0052884
G0050293
G0004771
G0034892
G0050021
G0050526
G0050357
G0102530
G0030339
G0052740
G0018733
G0102998
G0047741
G0003847
G0102293
G0090499
G0035560
G0047372
G0050033
G0050285
G0050349
G0047376
G0047658
G0047714
G0102259
G0102542
G0004623
G0047499
G0047498
G0052790
G0008970
G0061685
G0080030
G0052885
G0030599
G0046573
G0055042
G0102007
G0008806
G0033878
G0050490
G0102197
G0050527
G0017057
G0047554
G0080124
G0102209
G0004064
G0018732
G0050160
G

G0102990
G0034541
G0102117
G0102740
G0102820
G0102090
G0042086
G0030269
G0043776
G0140381
G0102094
G0102446
G0102718
G0034931
G0102215
G0043851
G0102941
G0102147
G0098614
G0102529
G0102101
G0102964
G0102608
G0008757
G0030738
G0004164
G0030734
G0030757
G0030776
G0030753
G0030772
G0030783
G0030787
G0035797
G0030769
G0030746
G0030580
G0008425
G0043430
G0043333
G0043428
G0043429
G0043334
G0030761
G0030790
G0052706
G0004481
G0046026
G0030794
G0030798
G0030737
G0004790
G0030779
G0030756
G0030733
G0030752
G0030775
G0030771
G0030780
G0030784
G0030788
G0030749
G0030745
G0030768
G0030741
G0102707
G0030760
G0008898
G0003838
G0008825
G0030791
G0030795
G0004851
G0018708
G0030736
G0030759
G0030778
G0030732
G0051742
G0030755
G0030774
G0030751
G0030770
G0030781
G0030785
G0046547
G0010420
G0044595
G0004395
G0030789
G0030748
G0030767
G0030744
G0030763
G0030740
G0102708
G0030792
G0008119
G0030796
G0030739
G0030758
G0030735
G0030754
G0030777
G0051741
G0030731
G0030750
G0030773
G0043839
G0030782
G0030786
G

...

G0006094
G0046370
G0033348
G0033350
G0046165
G1902653
G0034079
G0036197
G0006695
G0033490
G0033488
G0033489
G1902640
G0033021
G1902645
G0046173
G1901758
G0034312
G0042845
G0097040
G0034309
G0071271
G0046171
G1901961
G0046169
G1903175
G1901576
G0046307
G1901566
G0042236
G1901162
G0097055
G0042097
G1903605
G1903607
G0042225
G0042229
G0042239
G0048175
G0042235
G0045525
G0045526
G0050751
G0042398
G0150126
G0031108
G0042228
G0042231
G0042238
G0042234
G0042241
G0042091
G0042223
G0042227
G0045189
G0006023
G0006024
G0006025
G0042230
G0042253
G0042237
G0050757
G0042233
G0042983
G0045524
G0042094
G0042109
G0042222
G0050720
G0050719
G0042226
G0045350
G1902063
G1900620
G1902819
G1901503
G1901696
G1901441
G1902921
G1902923
G1902919
G0042619
G1902924
G0042621
G0009059
G1990145
G0042966
G0051767
G0042089
G0150156
G0042095
G0150154
G0045553
G0150137
G0042533
G0042033
G0042090
G0042240
G0045351
G0045349
G0045527
G0042232
G0070749
G0002375
G0000271
G0051070
G0035884
G0002378
G0002379
G0048153
G0010143
G

G0015793
G0051475
G0015778
G0006859
G0015797
G0015716
G0042916
G0015837
G0015841
G0015842
G1901374
G1901373
G0015833
G0002790
G0002776
G0002779
G0002782
G0061584
G0071993
G0042893
G0043213
G0042914
G0042915
G0006857
G0042938
G0042939
G0042884
G0042885
G0046968
G1905431
G0015834
G1901337
G1903415
G0006869
G0015918
G0030301
G0043691
G0033344
G0030299
G0070508
G0010879
G0035376
G0035627
G0034196
G0034197
G0098856
G0060752
G1990379
G0002494
G0010877
G0046864
G0046865
G0034633
G0046866
G0046867
G0019755
G0015840
G0015704
G0015895
G0015931
G0050657
G0051027
G0050658
G0051031
G0051030
G1990428
G0051029
G0051028
G0033227
G0006862
G0006864
G0015865
G0001408
G0051503
G0070729
G0015851
G0015855
G0035364
G0015856
G0006863
G0035344
G0015854
G0042906
G0015853
G0015850
G1901140
G0051937
G0048241
G0015874
G0015791
G0015799
G0015798
G0015731
G0006837
G0042909
G0015748
G1901571
G0015846
G0015844
G0042969
G0072337
G0015806
G0015849
G0006856
G0006811
G0006812
G0072511
G0070838
G0006829
G0015691
G0006816
G

G1902182
G0002930
G1905072
G0090008
G0007498
G0048389
G0048318
G0060429
G0072190
G0003313
G0007444
G0035220
G0035219
G0035218
G0035217
G0035216
G0035215
G0035214
G0035213
G0061180
G0003406
G0061053
G0060039
G0061071
G0072073
G0072163
G0061241
G0061232
G0061242
G0061272
G0061274
G0061275
G0061220
G0072164
G0072165
G0072177
G0072166
G0001657
G0072207
G0072243
G0072227
G0072234
G0072218
G0072229
G0072237
G0072236
G0072221
G0072233
G0072235
G0072286
G0072230
G0072225
G0072244
G0072264
G0072222
G0072170
G0072176
G0039022
G0072087
G0072009
G0072024
G0072010
G0072011
G0072080
G0072023
G0072021
G0072020
G0072019
G0072017
G0035777
G0072027
G0072014
G0035776
G0072025
G0072070
G0039020
G0072068
G0072067
G0021915
G0035050
G0001840
G0060428
G0060481
G0061144
G0061143
G0061326
G0072002
G0061327
G0061328
G0003158
G0061147
G0061448
G0072109
G0072223
G0061247
G0035989
G0060612
G0060611
G0060613
G0007503
G0007505
G0007504
G0072127
G0061285
G0072213
G0002932
G0003204
G0060427
G0003417
G1904817
G1904820
G

G0052077
G0052441
G0052279
G0052280
G0052281
G0052443
G0052283
G0052082
G0052284
G0052287
G0052533
G0052288
G0052532
G0052535
G0052289
G0052537
G0052538
G0052290
G0052539
G0052291
G0052292
G0052107
G0052030
G0033669
G0052293
G0052294
G0052295
G0052296
G0052297
G0052298
G0052299
G0052300
G0052301
G0052302
G0052060
G0003891
G0016000
G0015999
G0003893
G0008723
G0016448
G0003895
G0016449
G0003897
G0003898
G0000129
G0004036
G0009387
G0061505
G0008468
G0016660
G0019282
G0000505
G0047318
G0019109
G0019850
G0019851
G0042172
G0016400
G0004094
G0004093
G0016953
G0016952
G0008799
G0016225
G0008461
G0004101
G0048059
G0016809
G0004280
G0016510
G0034566
G0004330
G0008708
G0001513
G0004580
G0019106
G0008026
G0046971
G0043166
G0004406
G0004404
G0004403
G0004405
G0016315
G0001668
G0004472
G0016619
G0001718
G0070128
G0008702
G0047076
G0004524
G0008857
G0004537
G0016800
G0004562
G0016982
G0016686
G0016693
G0016685
G0016687
G0016224
G0008971
G0045126
G0042298
G0004646
G0018223
G0018224
G0018225
G0050222
G

G0035057
G0048051
G0048063
G0048064
G0048068
G0048141
G0048234
G1990925
G0048267
G0022621
G0048433
G0048413
G0048411
G0048429
G0048417
G0048421
G0048425
G0048434
G0048430
G0048418
G0048422
G0048426
G0048436
G0048435
G0048419
G0048420
G0048423
G0048424
G0048427
G0048428
G0048414
G0048415
G0048416
G0048410
G0048412
G0048431
G0048432
G0048157
G0009993
G0008099
G0016181
G0015619
G0043119
G0043118
G0051242
G0051243
G0009857
G0048547
G0048558
G0010251
G0048748
G0048594
G0048595
G0048828
G0042055
G0048676
G0016349
G0007456
G0021734
G0021733
G0033733
G0047607
G0019201
G0018790
G0000816
G0048253
G0016666
G0050475
G0050489
G0008698
G0008699
G0018552
G0008264
G0017068
G0030065
G0045432
G0045441
G0018691
G0000251
G0018540
G0004770
G0042319
G0045440
G0030236
G0050791
G0048552
G0051244
G2000021
G0042831
G0051869
G0055128
G0009592
G0043358
G0043357
G0045011
G0006444
G0070389
G2000823
G1904167
G1904169
G0034619
G0006010
G0019591
G1902593
G1902578
G0051227
G0051226
G0035786
G0007001
G0051277
G0051288
G

In [22]:
#test number of expected classes to be created by the algorithm above

c = 0

for k, v in nary.items():
    if v['parent'] == []:
        c+=1
    elif 'is_a' in [a['relation'] for a in v['parent']]:
        c+=1
        
print(c)

classes_not_created = []

for k, v in nary.items():
    if ('G' + k[3:]) not in GO_classes_created:
        classes_not_created.append(k)


42975


In [23]:
classes_not_created

[]

In [19]:
for x in classes_not_created:
    for a in nary[x]['parent']:
        if ('G' + a['id'][3:]) in GO_classes_created:
            with onto:
                nc = types.new_class(('G' + x[3:]), (getattr(onto, ('G' + a['id'][3:])),))
            recursive_classmaker_down(x)

In [25]:
len(list(onto.classes()))

42987

In [24]:
onto.save(file = "t3test2_no_semicolon_G", format = "rdfxml")